## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [233]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [234]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))


df_integrado = pd.read_csv("../../data/processed/datos_integrados_mc.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,0.0,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,0.0,True,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0,ok


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [235]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['completitud_fila','situacion_vivienda_ok']
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [236]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos ( filtrado a nivel de filas)

In [237]:
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,False,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,False,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,False,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,False,0.55,14.27,1,Y,ok


In [238]:
# Se filtran las filas con algún error detectado
print(f"filas antes del filtro:{df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]
#temp_c = temp[temp['antiguedad_empleado']<=50]
temp_c = temp[temp['regla_pct_ingresos']== 'ok']


print(f"filas despues del filtro:{temp_c.shape[0]}")

# También por las reglas del negocio
# df[df.regla_pct_ingresos == 'err']


filas antes del filtro:10127
filas despues del filtro:10108


Integración de datos

In [239]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"columnas despues del filtro:{temp_c.shape[0]}")

columnas despues del filtro:10108


In [240]:
print(f"columnas df_integrado despues del filtro:{df_integrado.shape[1]}")

columnas df_integrado despues del filtro:22


Transformación de atributos

In [241]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')
df_final = pd.concat([estado_civil_N, df_integrado], axis=1)
df_final.head()



# Columna: estado_CREDITO
cambios_estado_credito = {
    0 : 'C',
    1:'P'
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')
df_final = pd.concat([estado_credito_N, estado_civil_N,df_integrado], axis=1)
df_final.head()
# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_credito_N,estado_civil_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,P,C,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,ok,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0
1,C,C,768805383.0,21,1000,2,5.0,PROPIA,9600,False,0.10,11.14,0,N,ok,39.0,CASADO,ACTIVO,1144.0,M,12691.0,SECUNDARIO_COMPLETO,42.0,3.0
2,P,C,713982108.0,23,35000,2,4.0,ALQUILER,65500,False,0.53,15.23,1,N,ok,36.0,CASADO,ACTIVO,1887.0,M,3418.0,UNIVERSITARIO_COMPLETO,20.0,3.0
3,P,C,710821833.0,24,35000,4,8.0,ALQUILER,54400,False,0.55,14.27,1,Y,ok,54.0,CASADO,ACTIVO,1314.0,M,9095.0,DESCONOCIDO,26.0,1.0
4,P,N,769911858.0,21,2500,2,2.0,PROPIA,9900,False,0.25,7.14,1,N,ok,34.0,DESCONOCIDO,ACTIVO,1171.0,F,3313.0,SECUNDARIO_COMPLETO,20.0,4.0


In [242]:
df_final.shape

(10108, 24)

Atributos numericos que se discretizan

In [243]:
df_integrado['operaciones_ult_12m'].describe()

count    10108.000000
mean        64.892659
std         23.465252
min         10.000000
25%         45.000000
50%         67.000000
75%         81.000000
max        139.000000
Name: operaciones_ult_12m, dtype: float64

In [244]:
df_integrado.shape

(10108, 22)

In [245]:
# Antiguedad empleado
etiqueta_a_e =['menor 10','5_a_10','mayor_10']
rangos_a_e=[0,4,10,50 ]
valor_para_nan='NA'
antiguedad_empleados_N= pd.cut(df_integrado['antiguedad_empleado'],
                               bins=rangos_a_e,
                               labels=etiqueta_a_e,
                               right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

#edad

etiquetas_e=['menor 25','25_a_30']
rangos_e=[0,24,50]
edad_N=pd.cut(df_integrado['edad'],
              bins=rangos_e,
              labels=etiquetas_e)

edad_N.value_counts()

#pct_ingreso

etiquetas_p_i=['hata_20','20_a_40','40_a_60','mayor_60']
rangos_p_i=[0,0.19,0.39,0.60,0.99]
regla_pct_ingresos_N=pd.cut(df_integrado['pct_ingreso'],
                            bins=rangos_p_i,
                            labels=etiquetas_p_i)
regla_pct_ingresos_N.value_counts()

#ingresos
etiqueta_i =['hasta_20k','20k_a_50k','50k_a_100k','mayor_100k']
rangos_i=[0,19999,49999,99999,999999]
ingresos_N=pd.cut(df_integrado['ingresos'],
                  bins=rangos_i,
                  labels=etiqueta_i)

ingresos_N.value_counts()

#tasa_interes

etiquetas_t_i=['hasta_7p','7p_a_15p','15p_a_20p','mayor_20p']
rangos_t_i=[0,6.99,14.99,19.99,100]
tasa_interes_N=pd.cut(df_integrado['tasa_interes'],
                      bins=rangos_t_i,
                      labels=etiquetas_t_i)

tasa_interes_N.value_counts()


#antiguedad_cliente
etiquetas_a_c=['menor_2y','2y_a_4y','mayor_4y']
rangos_a_c=[0,24,48,100]
antiguedad_cliente_N=pd.cut(df_integrado['antiguedad_cliente'],
                            bins=rangos_a_c,
                            labels=etiquetas_a_c)
antiguedad_cliente_N.value_counts()

#limite_credito
etiquetas_l_tc=['menor_3k','3k_a_5k','5k_a_10k','mayor_10k']
rangos_l_tc=[0,2999,4999,9999,10000]
limite_credito_tc_N=pd.cut(df_integrado['limite_credito_tc'],
                        bins=rangos_l_tc,
                        labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

#gastos_ultimos_12m

etiquetas_g_u12=['menor_1k','1k_a_4k','4k_a_6k','6k_a_8k','8k_a_10k','mayor_10k']
rangos_g_u12=[0,999,3999,5999,7999,9999,10000]
gastos_ult_12m_N=pd.cut(df_integrado['gastos_ult_12m'],
                        bins=rangos_g_u12,
                        labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()



#operaciones_ult_12m
etiquetas_o_u12=['menor_15k','15k_a_30k','30k_a_50k','50k_a_75k','75k_a_100k','mayor_100k']
rangos_o_u12=[0,14,29,49,74,99,100]
operaciones_ult_12m_N=pd.cut(df_integrado['operaciones_ult_12m'],
                        bins=rangos_o_u12,
                        labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50k_a_75k     3393
75k_a_100k    3004
30k_a_50k     2413
15k_a_30k      587
mayor_100k      38
menor_15k       24
Name: count, dtype: int64

In [246]:
df_integrado.shape

(10108, 22)

In [247]:
col_eliminar_final=[
    'edad',
    'antiguedad_empleado',
    'antiguedad_cliente',
    'ingresos',
    'pct_ingreso',
    'tasa_interes',
    'regla_pct_ingresos',
    'gastos_ult_12m',
    'operaciones_ult_12m',
    'id_cliente',
    'estado_credito',
    'estado_civil',
  
]

df_integrado.drop(col_eliminar_final,inplace=True,axis=1)

df_final=pd.concat([edad_N,antiguedad_empleados_N,antiguedad_cliente_N,ingresos_N,regla_pct_ingresos_N,tasa_interes_N,
 gastos_ult_12m_N,operaciones_ult_12m_N,estado_civil_N,estado_credito_N,df_integrado],axis=1)

In [248]:
df_final.head()

,edad,antiguedad_empleado,antiguedad_cliente,ingresos,pct_ingreso,tasa_interes,gastos_ult_12m,operaciones_ult_12m,estado_civil_N,estado_credito_N,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,estado_cliente,genero,limite_credito_tc,nivel_educativo,personas_a_cargo
0,menor 25,NA,2y_a_4y,50k_a_100k,40_a_60,15p_a_20p,1k_a_4k,15k_a_30k,C,P,35000,3,ALQUILER,False,Y,ACTIVO,M,4010.0,UNIVERSITARIO_COMPLETO,2.0
1,menor 25,5_a_10,2y_a_4y,hasta_20k,hata_20,7p_a_15p,1k_a_4k,30k_a_50k,C,C,1000,2,PROPIA,False,N,ACTIVO,M,12691.0,SECUNDARIO_COMPLETO,3.0
2,menor 25,5_a_10,2y_a_4y,50k_a_100k,40_a_60,15p_a_20p,1k_a_4k,15k_a_30k,C,P,35000,2,ALQUILER,False,N,ACTIVO,M,3418.0,UNIVERSITARIO_COMPLETO,3.0
3,menor 25,5_a_10,mayor_4y,50k_a_100k,40_a_60,7p_a_15p,1k_a_4k,15k_a_30k,C,P,35000,4,ALQUILER,False,Y,ACTIVO,M,9095.0,DESCONOCIDO,1.0
4,menor 25,menor 10,2y_a_4y,hasta_20k,20_a_40,7p_a_15p,1k_a_4k,15k_a_30k,N,P,2500,2,PROPIA,False,N,ACTIVO,F,3313.0,SECUNDARIO_COMPLETO,4.0


In [249]:
import dtale as dt
informe =dt.show(df_final)
informe.open_browser()

/home/sebastian/.local/lib/python3.10/site-packages/dtale/column_analysis.py:403: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

/home/sebastian/.local/lib/python3.10/site-packages/dtale/column_analysis.py:403: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

/home/sebastian/.local/lib/python3.10/site-packages/dtale/column_analysis.py:403: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

/home/sebastian/.local/lib/python3.10/site-packages/dtale/column_analysis.py:403: FutureWarning:

pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.

/home/sebastian/.local/lib/python3.10/site-packages/dtale/column_analysis.py:403: FutureWarning:

pandas.value_counts is deprecated 

In [250]:
df_final.to_csv("../../data/final/datos_finales.csv",sep=';',index=False)